This program purpose is to scrap job listing for specific locations and specific job titles or categories.
We will scrap job listing from linkedin


In [34]:
"""
Installing the packages we need
"""
%pip install beautifulsoup4
%pip install requests
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached numpy-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached numpy-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.1 MB)
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)
Note: you may need to restart the kernel to use updated packages.


In [18]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
keywords = 'developer'
location = 'US'
sample_job_listing_url = f"https://www.linkedin.com/jobs/search?keywords={keywords}&location={location}"
job_post_list = []
counter = 0
for i in range(0, 1):
    """
    Getting list of job posting by searching for <li> element
    """
    counter += 25
    current_page_listing_url = f"{sample_job_listing_url}&start={counter}"
    response = requests.get(sample_job_listing_url)
    list_data = response.text
    # Parsing the HTML response
    list_soup = BeautifulSoup(list_data, 'html.parser')
    page_jobs = list_soup.find_all('li')
    print(f"Page {i+1} has {len(page_jobs)} jobs")
    print(
        f'Fetching jobs, start: {counter} for url: {current_page_listing_url}')
    job_id_list = []
    # Extracting information from each job
    for job in page_jobs:
        """
        For each <li> we get the job id
        Example job post data-entity-urn="urn:li:jobPosting:4179181538"
        """
        base_card_div = job.find("div", {"class": "base-card"})
        if base_card_div is None:
            continue
        # This splitting might not work if they change the format of the job_id
        job_id = base_card_div.get('data-entity-urn').split(":")[3]
        job_id_list.append(job_id)
        """
        Now that we have the job ids , we are going to fetch the job details
        Sample URL: https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4179518973
        """
    print(f"job_id_list length: {len(job_id_list)}")
    """
    Once we get all the job ids, we start fetching the job details
    """
    for job_id in job_id_list:
        print(f"Fetching job details for job_id: {job_id}")
        job_details_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
        print(f"Fetch url: {job_details_url}")
        response = requests.get(job_details_url)
        job_soup = BeautifulSoup(response.text, 'html.parser')
        job_post = {}
        # Getting company name
        try:
            company_name = job_soup.find(
                "a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
        except:
            company_name = None
            continue
        job_post['company_name'] = company_name
        # Getting position name
        try:
            position_name = job_soup.find(
                "h2", {"class": "topcard__title"}).text.strip()
        except:
            position_name = ''
            continue
        job_post['position_name'] = position_name
        # Getting published date
        try:
            posted_time_ago = job_soup.find(
                "span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
        except:
            posted_time_ago = ''
        job_post['posted_time_ago'] = posted_time_ago
        job_post['web scrapped date'] = datetime.now().strftime(
            "%Y-%m-%d %H:%M:%S")
        # Getting number of applicants
        try:
            num_applicants = job_soup.find(
                "figcaption", {"class": "num-applicants__caption"}).text.strip()
        except:
            num_applicants = ''
        job_post['num_applicants'] = num_applicants

        # Getting job full description html
        try:
            job_description_html = job_soup.find(
                "div", {"class": "decorated-job-posting__details"})
            job_description_text = job_description_html.find(
                "div", {"class": "show-more-less-html__markup"})
        except:
            print(f"Error skipping because not job description found: {job_id}")
            continue
        job_post['job_description_html'] = job_description_html
        job_post['job_description_text'] = job_description_text
        job_post['job_id'] = job_id
        job_post_list.append(job_post)
print(f"Total jobs fetched: {len(job_post_list)}")
job_list_df = pd.DataFrame(job_post_list)
if len(job_list_df) > 0:
    job_list_df.to_csv(
    f"keywords: {keywords}, location: {location}, date: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}.csv", index=False)

Page 1 has 186 jobs
Fetching jobs, start: 25 for url: https://www.linkedin.com/jobs/search?keywords=developer&location=US&start=25
job_id_list length: 60
Fetching job details for job_id: 4176419609
Fetch url: https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4176419609
Fetching job details for job_id: 4143812266
Fetch url: https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4143812266
Fetching job details for job_id: 4144718777
Fetch url: https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4144718777
Fetching job details for job_id: 4165606482
Fetch url: https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4165606482
Fetching job details for job_id: 4130642730
Fetch url: https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4130642730
Fetching job details for job_id: 4177383455
Fetch url: https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4177383455
Fetching job details for job_id: 4170327264
Fetch url: https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/417